In [ ]:
# Chargement des dépendances

# Bibliothèques pour le traitement de texte et le langage naturel
import spacy

# Bibliothèques pour le calcul et la manipulation de données
import numpy as np  # Manipulation de tableaux et de matrices
import pandas as pd  # Manipulation et analyse de données avec des DataFrames

# Bibliothèques pour la visualisation de données
import seaborn as sns  # Visualisation de données
from wordcloud import WordCloud  # Génération de nuages de mots
import matplotlib.pyplot as plt  # Création de graphiques et de visualisations
plt.style.use('ggplot')  # Application d'un style de visualisation (ggplot)

# Bibliothèques pour l'apprentissage automatique
from keras.models import Sequential  # Modèle séquentiel pour les réseaux de neurones
from keras.layers import LSTM, Dense, Embedding, Dropout  # Couches pour le réseau de neurones
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer # Transformation de texte en séquences de nombres
from keras.preprocessing.sequence import pad_sequences  # Normalisation des longueurs de séquences
from scikeras.wrappers import KerasClassifier  # Intégration de Keras dans scikit-learn

# Bibliothèques pour l'évaluation et la validation des modèles
from sklearn.model_selection import train_test_split, cross_val_score  # Division des données et validation croisée
from sklearn.metrics import classification_report, confusion_matrix  # Évaluation des performances des modèles

# Utilitaires
import re  # Utilisé pour les opérations avec des expressions régulières
import time  # Utilisé pour des opérations liées au temps
import warnings  # Pour gérer les avertissements
warnings.filterwarnings("ignore")  # Ignore les avertissements pour garder la sortie propre


In [ ]:
!python -m spacy download fr_core_news_sm

In [ ]:
# Charger le modèle français
nlp = spacy.load('fr_core_news_sm')

In [ ]:
! pip freeze > requirements.txt

In [ ]:
# Charger le dataset
dataset = pd.read_csv('dataset_french.csv')
dataset = dataset.sample(frac=1).reset_index(drop=True)  # Mélanger les données

In [ ]:
# Analyse exploratoire du dataset
print("Aperçu du dataset :")
dataset.head()  # Affiche les 5 premières lignes


In [ ]:
print("\nInformations sur le dataset :")
dataset.info()  # Informations sur les types de données


In [ ]:
# Statistiques descriptives pour les colonnes numériques
print("\nStatistiques descriptives :")
dataset.describe()

In [ ]:
# Visualisation des longueurs de tweets
plt.figure(figsize=(10, 6))
dataset['text_length'] = dataset['text'].str.len()  # Ajouter une colonne pour la longueur des tweets
dataset['text_length'].plot.hist(bins=30, alpha=0.7)  # Histogramme des longueurs de tweets
plt.title('Distribution de la longueur des tweets')
plt.xlabel('Longueur du tweet')
plt.ylabel('Fréquence')
plt.show()

In [ ]:
# Comptage des classes de sentiment avec un graphique à barres
plt.figure(figsize=(8, 6))
dataset['label'].value_counts().plot.bar()  # Compte des classes de sentiment
plt.title('Comptage des classes de sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Nombre de tweets')
plt.xticks(rotation=0)  # Rotation des étiquettes pour une meilleure lisibilité
plt.show()

In [ ]:
dataset.columns

In [ ]:
# Garder uniquement les colonnes nécessaires
dataset = dataset[['label', 'text']]

In [ ]:
dataset.head()

In [ ]:
np.sum(dataset.isnull().any(axis=1))

In [ ]:
dataset.dropna(inplace=True)

## Prétraitement des tweets


In [ ]:
# Suppression des URL, mentions et hashtags
def remove_urls_mentions_hashtags(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Supprimer les URL
    text = re.sub(r'@\w+', '', text)  # Supprimer les mentions
    text = re.sub(r'#\w+', '', text)  # Supprimer les hashtags
    return text

In [ ]:
# Mise en minuscules
def to_lowercase(text):
    return text.lower()

In [ ]:
# Lemmatisation avec WordNetLemmatizer
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_tokens = [
        token.lemma_ for token in doc if not token.is_stop and not token.is_punct
    ]
    return ' '.join(lemmatized_tokens)

In [ ]:
# Appliquer chaque étape de prétraitement
dataset['text_cleaned'] = dataset['text']
dataset['text_cleaned'] = dataset['text_cleaned'].apply(remove_urls_mentions_hashtags)
dataset['text_cleaned'] = dataset['text_cleaned'].apply(to_lowercase)
dataset['text_cleaned'] = dataset['text_cleaned'].apply(lemmatize_text)

In [ ]:
dataset.head()

In [ ]:
# Visualisation avec WordCloud
plt.figure(figsize=(12, 8))
wordcloud = WordCloud(width=2000, height=800, background_color='white').generate(' '.join(dataset['text_cleaned']))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nuage de mots des tweets')
plt.show()

In [ ]:
# Tokenisation et Padding
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(dataset['text_cleaned'].values)
X = tokenizer.texts_to_sequences(dataset['text_cleaned'].values)
X = pad_sequences(X, maxlen=100)
X[:7]

In [ ]:
X.shape

In [ ]:
# Enregistrer le tokenizer
joblib.dump(tokenizer, 'tokenizer_french.pkl')

In [ ]:
# Encodage des étiquettes
y = pd.get_dummies(dataset['label']).values
[print(dataset['label'][i], y[i]) for i in range(0, 7)]

In [ ]:
# Division des données en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Construction du modèle LSTM
def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=100000, output_dim=256, input_length=100))
    model.add(Dropout(0.5))
    model.add(LSTM(512, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
    model.add(LSTM(512, dropout=0.5, recurrent_dropout=0.2))
    model.add(Dense(units=3, activation='softmax'))

    # Compilation du modèle
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Création du modèle LSTM
model = create_model()

In [ ]:
model.summary()

## Entrainement du modèle

In [ ]:
print("Shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# Définir des callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_french.keras', save_best_only=True)

In [ ]:
# Entraîner le modèle
model.fit(
    X_train,
    y_train,
    epochs=20,  # Vous pouvez ajuster ce nombre
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1,
    callbacks=[early_stopping, model_checkpoint]
)


In [ ]:
dataset.to_csv('tweets.csv', index=False)

In [ ]:
# Évaluation du modèle
score, acc = model.evaluate(X_test, y_test, verbose=1, batch_size=32)
print(f"Test score: {score}")
print(f"Test accuracy: {acc}")

In [ ]:
# Prédictions et rapport de classification
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
# Rapport de classification
print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Normaliser par ligne pour obtenir des pourcentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Définir les étiquettes
labels = ['négatif', 'neutre', 'positif']

# Tracer la matrice de confusion
sns.heatmap(cm_percentage, annot=True, fmt='.2f', cmap='Blues', xticklabels=labels, yticklabels=labels)
# Ajouter le signe % dans les annotations
for text in plt.gca().texts:
    text.set_text(f"{text.get_text()}%")
plt.show()
